In [1]:
%run StdPackages.ipynb
os.chdir(d['py'])
from GmsPy import *
os.chdir(d['curr'])

No clean-up of work-folder


In [2]:
def read(x,path = d['gams']):
    with open(os.path.join(path,x), 'r') as f:
        return f.read()

*Load test database and groups*

In [3]:
db = GpyDB(db = os.path.join(d['data'], 'AbatementData.gdx'))
with open(os.path.join(d['data'], 'AbatementGroups'), "rb") as file:
    c = pickle.load(file)

# ```GmsPy.GmsSettings```

*A class of settings used to specify gams models.*

The class is initialized either with kwargs or from a pickle. If it is not initialized from a pickle, a number of default attributes are defined.

```python
    class GmsPy.GmsSettings:
        def __init__(self, file_path = None, **kwargs):
```
If a file path is not supplied, the kwargs are merged with default settings. Default settings:
* ```self.name:``` Name of the settings file (a string).
* ```self.macros:``` Empty dictionary. A container to capture the declared macros (GAMS macros).
* ```self.Precompiler:``` An instance of the Precompiler class from the ```dreamtools``` package (gamY).
* ```self.Compile:``` A user-class defined to write gams code from group/model specifications. 
* ```self.db:``` A ```GpyDB``` database instance. 
* ```self.locals:``` Dictionary with local variables to be added as placeholders when running the gams model.
* ```self.states:``` Dictionary with different model states (e.g. different selection of endogenous groups, blocks etc.).
* ```self.state:``` The current state of the model (key in self.states).

*Initialize test settings:*

In [4]:
s = GmsSettings(**{'name': 'TestSettings', 'db': db, 'Compile': c})
s.Precompiler.locals = dict(techtype="'logit'")

## Main attributes:

#### ```self.name```

The name of the settings file is used a lot as the default option. 
* All ```states``` (see explanation later) that we define the model over is per construction called ```self.name_state.```. The default ```state``` is called ```B``` for baseline. This name ```self.name_state``` is also used as the default model name.
* The name is used as a prefix to the ```self.args``` that are collected (see explanation later). This is done to make merging with other settings files easier.

In [5]:
s.states # the name includes prefix 'TestSettings'

{'B': {'name': 'TestSettings_B',
  'g_endo': <pyDatabases._mixedTools.OrdSet at 0x24b2c841a60>,
  'g_exo': <pyDatabases._mixedTools.OrdSet at 0x24b2b096f40>,
  'blocks': <pyDatabases._mixedTools.OrdSet at 0x24b2b091460>,
  'solve': None,
  'args': {},
  'text': {}}}

In [6]:
s.stdArgs().keys() # the args includes prefix 'TestSettings'

dict_keys(['Root', 'TestSettings_Functions', 'TestSettings_Declare', 'TestSettings_B_Fix', 'TestSettings_B_Unfix', 'TestSettings_B_Model', 'TestSettings_B_Solve'])

#### ```self.macros```

A dictionary used to capture GAMS macros. Keys indicate name of macros (```F``` in the example below) and values indicate the text used for the macros. Macros are declared either in files or strings using the syntax:

```$MACRO F(x,y) (sum(n, x) * y)```

When the ```GmsSettings``` compiles the text used to run the GAMS model, we can add the argument ```functions``` (string) - and the compiler adds the macros to ```self.macros```.

#### ```self.Precompiler```

An instance of the ```dreamtools.gamY.Precompiler``` class. This class uses a slightly augmented version of the Precompiler that processes strings and files. Amongst other things, we use this to define model blocks and models. An auxiliary function ```GmsPy.arg2string(x,t=None)``` allows us to pass arguments using the ```Precompiler``` in various ways.

#### ```self.Compile```

An instance of the ```GmsPy.Compile``` class. Used to handle group definitions of variables (see relevant documentation).

#### ```self.db```

The main database used throughout.

### Test runs:

In [7]:
s.Compile.run()

Specify functions and blocks:

In [8]:
functions = read('AbatementFunctions.gms') # read in file that declares functions
blocks = read('AbatementBlocks.gms') # read in equations for model.

Specify the endogenous/exogenous groups, specify blocks of equations:

In [9]:
s['g_endo'] = ['G_V01_NT_endo_always', 'G_V01_NT_endo_base', 'G_V01_T_endo_always', 'G_V01_T_endo_base', 'G_V01_ACC_endo_base']
s['g_exo'] = ['G_V01_NT_exo_always', 'G_V01_NT_exo_base', 'G_V01_T_exo_always', 'G_V01_T_exo_base']
s['blocks'] = ['M_V01_NT', 'M_V01_T_always', 'M_V01_T_base', 'M_V01_ACC']

We can now collect a standard selection of arguments:

In [10]:
s['args'] = s.stdArgs(blocks=blocks,functions=functions)
s['args'].keys()

dict_keys(['Root', 'TestSettings_Functions', 'TestSettings_Declare', 'TestSettings_Blocks', 'TestSettings_B_Fix', 'TestSettings_B_Unfix', 'TestSettings_B_Model', 'TestSettings_B_Solve'])

To solve the model, we simply use the ```self.write``` method again:

In [11]:
%%capture
s.write()

And set up model and solve:

In [12]:
m = GmsPy.GmsModel(ws=d['work'],**{'cns': 'CONOPT4'}) # use CONOPT4 to solve CNS models.
m.addlocal(db.name, os.path.join(d['data'], 'AbatementData.gdx'))
m.run(run = '\n'.join(s['text'].values()))